# embedding_processor 테스트

In [1]:
# 환경 설정 및 라이브러리 임포트
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# 작업 디렉토리를 프로젝트 루트로 변경 (config.json 찾기 위해)
os.chdir(project_root)

from src.config import get_config
from src.db import DocumentsDB, EmbeddingsDB
from src.processors.embedding_processor import EmbeddingProcessor
from src.utils.logging_config import get_logger

logger = get_logger(__name__)
config = get_config()

print("✓ 라이브러리 임포트 완료")
print(f"프로젝트 루트: {project_root}")
print(f"현재 작업 디렉토리: {Path.cwd()}")
print(f"설정 버전: {config.version}")

✓ 설정 파일 로드 완료: config/config.json
✓ 설정 검증 통과
✓ 라이브러리 임포트 완료
프로젝트 루트: d:\GoogleDrive\codeit_ai_g2b_search
현재 작업 디렉토리: d:\GoogleDrive\codeit_ai_g2b_search
설정 버전: 1.0.0


In [2]:
# 모듈 리로드 (코드 변경 시 재실행)
import importlib
import src.db.embeddings_db
importlib.reload(src.db.embeddings_db)
from src.db import EmbeddingsDB

print("✓ EmbeddingsDB 모듈 리로드 완료")

✓ EmbeddingsDB 모듈 리로드 완료


## 1. DocumentsDB 조회 테스트
문서 데이터베이스에서 파일 정보와 페이지 데이터를 조회합니다.

In [3]:
# DocumentsDB 초기화
docs_db = DocumentsDB()

# 문서 통계 조회
stats = docs_db.get_document_stats()
print("=== 문서 통계 ===")
for key, value in stats.items():
    print(f"{key}: {value}")

# 파일 목록 조회 (상위 5개)
with docs_db._get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT file_hash, file_name, total_pages FROM file_info LIMIT 5")
    files = cursor.fetchall()
    
print("\n=== 파일 목록 (상위 5개) ===")
for f in files:
    print(f"Hash: {f['file_hash'][:16]}... | Name: {f['file_name']} | Pages: {f['total_pages']}")

=== 문서 통계 ===
total_files: 2
total_pages: 6
total_tokens: 678
total_size_bytes: 1715
total_size_mb: 0.0

=== 파일 목록 (상위 5개) ===
Hash: 88d7f8d884832042... | Name: 공공데이터_품질관리_가이드라인_2024.pdf | Pages: 3
Hash: de78b810c7feaaaa... | Name: AI_학습용_데이터_구축_지침서_v2.pdf | Pages: 3


## 2. EmbeddingsDB CRUD 테스트
임베딩 데이터베이스의 추가, 조회, 수정, 삭제 기능을 테스트합니다.

In [4]:
# EmbeddingsDB 초기화
emb_db = EmbeddingsDB()

# 테스트용 데이터
test_embedding_hash = "test_embed_hash_001"
test_file_hash = "test_file_hash_001"

# 1. INSERT 테스트 - 임베딩 메타데이터 추가
print("=== 1. INSERT 테스트 ===")
emb_db.insert_embedding_meta(
    embedding_hash=test_embedding_hash,
    file_hash=test_file_hash,
    chunk_size=600,
    chunk_overlap=100,
    preprocessing_option={},
    embedding_model="text-embedding-3-small",
    total_chunks=10,
    vector_path=f"data/vectorstore/{test_embedding_hash}.faiss"
)
print(f"✓ 임베딩 메타데이터 추가 완료: {test_embedding_hash}")

# 2. SELECT 테스트 - 조회
print("\n=== 2. SELECT 테스트 ===")
with emb_db._get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM embedding_meta WHERE embedding_hash = ?", (test_embedding_hash,))
    result = cursor.fetchone()
    if result:
        print(f"조회 성공:")
        for key in result.keys():
            print(f"  {key}: {result[key]}")
    else:
        print("조회 실패")

# 3. 청크 매핑 추가
print("\n=== 3. 청크 매핑 추가 ===")
for i in range(3):
    emb_db.insert_chunk_mapping(
        embedding_hash=test_embedding_hash,
        file_hash=test_file_hash,
        file_name="test_document.pdf",
        chunk_text=f"테스트 청크 텍스트 {i+1}",
        vector_index=i,
        estimated_tokens=50
    )
print(f"✓ {3}개 청크 매핑 추가 완료")

# 4. 청크 조회
print("\n=== 4. 청크 조회 ===")
with emb_db._get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute(
        "SELECT chunk_id, chunk_text, vector_index FROM chunk_mapping WHERE embedding_hash = ?",
        (test_embedding_hash,)
    )
    chunks = cursor.fetchall()
    for chunk in chunks:
        print(f"  ID: {chunk['chunk_id']} | Index: {chunk['vector_index']} | Text: {chunk['chunk_text'][:50]}...")

# 5. DELETE 테스트 - 삭제
print("\n=== 5. DELETE 테스트 ===")
with emb_db._get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("DELETE FROM embedding_meta WHERE embedding_hash = ?", (test_embedding_hash,))
    conn.commit()
print(f"✓ 테스트 데이터 삭제 완료: {test_embedding_hash}")

=== 1. INSERT 테스트 ===
✓ 임베딩 메타데이터 추가 완료: test_embed_hash_001

=== 2. SELECT 테스트 ===
조회 성공:
  embedding_hash: test_embed_hash_001
  file_hash: test_file_hash_001
  chunk_size: 600
  chunk_overlap: 100
  preprocessing_option: {}
  embedding_model: text-embedding-3-small
  total_chunks: 10
  vector_path: data/vectorstore/test_embed_hash_001.faiss
  created_at: 2025-11-08 13:50:34
  updated_at: 2025-11-08 22:50:34.916927

=== 3. 청크 매핑 추가 ===
✓ 3개 청크 매핑 추가 완료

=== 4. 청크 조회 ===
  ID: 1 | Index: 0 | Text: 테스트 청크 텍스트 1...
  ID: 2 | Index: 1 | Text: 테스트 청크 텍스트 2...
  ID: 3 | Index: 2 | Text: 테스트 청크 텍스트 3...

=== 5. DELETE 테스트 ===
✓ 테스트 데이터 삭제 완료: test_embed_hash_001


## 3. EmbeddingProcessor 클래스 동작 테스트
실제 문서 파일로 임베딩 프로세서의 전체 파이프라인을 테스트합니다.

In [5]:
# 실제 임베딩 처리 실행 (주석 해제하여 실행)
# 주의: OpenAI API 비용이 발생합니다!

# if file_info:
#     embedding_hash = processor.process_document(file_hash)
#     
#     if embedding_hash:
#         print(f"\n✓ 임베딩 처리 완료!")
#         print(f"임베딩 해시: {embedding_hash[:16]}...")
#         
#         # 생성된 임베딩 메타데이터 확인
#         with emb_db._get_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute(
#                 "SELECT * FROM embedding_meta WHERE embedding_hash = ?",
#                 (embedding_hash,)
#             )
#             meta = cursor.fetchone()
#             if meta:
#                 print("\n=== 임베딩 메타데이터 ===")
#                 for key in meta.keys():
#                     print(f"  {key}: {meta[key]}")
#         
#         # 청크 매핑 확인
#         with emb_db._get_connection() as conn:
#             cursor = conn.cursor()
#             cursor.execute(
#                 "SELECT COUNT(*) as count FROM chunk_mapping WHERE embedding_hash = ?",
#                 (embedding_hash,)
#             )
#             chunk_count = cursor.fetchone()['count']
#             print(f"\n총 청크 수: {chunk_count}")
#     else:
#         print("✗ 임베딩 처리 실패")

print("주석을 해제하고 실행하세요.")

주석을 해제하고 실행하세요.


In [6]:
# EmbeddingProcessor 초기화
processor = EmbeddingProcessor()

print("=== EmbeddingProcessor 설정 ===")
print(f"청크 크기: {processor.chunk_size}")
print(f"청크 중첩: {processor.chunk_overlap}")
print(f"임베딩 모델: {processor.embedding_model}")

# DocumentsDB에서 첫 번째 파일 해시 가져오기
with docs_db._get_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT file_hash, file_name FROM file_info LIMIT 1")
    file_info = cursor.fetchone()

if file_info:
    file_hash = file_info['file_hash']
    file_name = file_info['file_name']
    print(f"\n테스트 대상 파일: {file_name}")
    print(f"파일 해시: {file_hash[:16]}...")
    
    # 임베딩 처리 (주의: OpenAI API 키 필요)
    print("\n⚠ OpenAI API 키가 설정되어 있어야 합니다.")
    print("임베딩 처리를 시작하려면 다음 셀에서 process_document()를 호출하세요.")
else:
    print("⚠ DocumentsDB에 파일이 없습니다. 먼저 문서를 처리하세요.")

2025-11-08 22:50:35 [E] src.processors.embedding_processor - LangChain이 설치되지 않았습니다.
=== EmbeddingProcessor 설정 ===
청크 크기: 600
청크 중첩: 100
임베딩 모델: text-embedding-3-small

테스트 대상 파일: 공공데이터_품질관리_가이드라인_2024.pdf
파일 해시: 88d7f8d884832042...

⚠ OpenAI API 키가 설정되어 있어야 합니다.
임베딩 처리를 시작하려면 다음 셀에서 process_document()를 호출하세요.
